In [ ]:
import nest_asyncio
nest_asyncio.apply()
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from pydantic_ai.models.groq import GroqModel
import json
model= GroqModel('llama-3.1-8b-instant')
# agent = Agent(model, system_prompt='Be concise, reply with one sentence.')

# result = agent.run_sync('Where does "hello world" come from?')
# print(result.output)


In [ ]:
ticket1= {
    "ticket_id": "SUP-001",
    "customer_tier": "free",
    "subject": "This product is completely broken!!!",
    "message": "Nothing works! I can't even log in. This is the worst software I've ever used.",
    "previous_tickets": 0,
    "monthly_revenue": 0,
    "account_age_days": 2
}

class TicketAnalysis(BaseModel):
    summarized_issue: str= Field(description="summarize the issue in short")
    sentiment: str= Field(description="Sentiment tone from subject/message: (positive,or neutral, or negative)")
    urgency_level: str= Field(description="Urgency level: Critical/P1, High/P2, Medium/P3, Low/P4")
    monthly_revenue: int
    account_age_days: int

agent= Agent(
    model=model,
    output_model=TicketAnalysis,
    system_prompt=(
        "You are a support ticket analyzer.\n"
        "Given a ticket with subject, message, revenue, and account age, perform the following:\n"
        "- Summarize the core issue in one short sentence explaining the issue to support.\n"
        "- Analyze the customer's tone to determine the sentiment (positive,or neutral, or negative).\n"
        "- Based on tone and message content, assess the urgency level as one of: Critical/P1, High/P2, Medium/P3, Low/P4. Only Select One\n"
        "Be concise and well structured and do not provied reasons for Tone and Urgency Level."
    )
)

result = agent.run_sync(str(ticket1))


In [81]:
print(result.output)

**Ticket Summary:** The customer is experiencing severe issues with login functionality and reports the product as being completely broken.

**Tone Sentiment:** Negative

**Urgency Level:** Critical/P1


In [ ]:
import json
json_output= result.output.model_dump()
print(json.dumps(json_output, indent=2))

AttributeError: 'str' object has no attribute 'model_dump'

In [ ]:
from pydantic import BaseModel, Field

ticket1= {
"ticket_id": "12345",
"customer_tier": "premium", # free/premium/enterprise
"subject": "API returning 500 errors intermittently",
"message": "Hi, our production system has been failing",
"previous_tickets": 3,
"monthly_revenue": 5000,
"account_age_days": 450
}

class TicketAnalysis(BaseModel):
    summarized_issue: str= Field(description="summarize the issue in short")
    sentiment: str= Field(description="Sentiment tone from subject/message: (positive,or neutral, or negative)")
    urgency_level: str= Field(description="Urgency level: Critical/P1, High/P2, Medium/P3, Low/P4")
    monthly_revenue: int
    account_age_days: int
    previous_tickets: int
    ticket_id: int
    customer_tier: str

model= GroqModel("llama-3.1-8b-instant")
agent= Agent(
    model=model,
    #output_model=TicketAnalysis,
    output_type=TicketAnalysis,
    system_prompt=(
        "You are a support ticket analyzer.\n"
        "Given a ticket with subject, message, revenue, and account age, perform the following:\n"
        "- Summarize the core issue in one short sentence explaining the issue to support.\n"
        "- Analyze the customer's tone to determine the sentiment (positive,or neutral, or negative).\n"
        "- Based on tone and message content and customer tier, assess the urgency level as one of: Critical/P1, High/P2, Medium/P3, Low/P4. Only Select One\n"
        "Be concise and well structured and do not provide reasons for Tone and Urgency Level."
        "Return ONLY a JSON object with keys: summarized_issue, sentiment, urgency_level, monthly_revenue, account_age_days, previous_tickets, ticket_id, customer_tier."
    )
)

result= agent.run_sync(str(ticket1))

json_output= result.output.model_dump()
print(json.dumps(json_output, indent=2))

{
  "summarized_issue": "API returning 500 errors intermittently",
  "sentiment": "negative",
  "urgency_level": "Critical/P1",
  "monthly_revenue": 5000,
  "account_age_days": 450,
  "previous_tickets": 3,
  "ticket_id": 12345,
  "customer_tier": "premium"
}


In [ ]:
class IssueTypeOutput(BaseModel):
    issue_type: str= Field(
        description="The category of the issue such as: bug, feature_request, billing, security, login_issue, ui_feedback, documentation, general_query"
    )
    confidence_score: float= Field(
        description="A float between 0 and 1 representing confidence in classification"
    )

model = GroqModel("llama-3.1-8b-instant")

issue_classifier_agent = Agent(
    model=model,
    output_model=IssueTypeOutput,
    system_prompt=(
        "You are an issue classification expert for customer support.\n"
        "Based on the 'subject' and 'message' of a support ticket, classify the issue type.\n"
        "Use one of the following categories ONLY: bug, feature_request, billing, security, login_issue, ui_feedback, documentation, general_query.\n"
        "Return the issue type and a confidence score between 0 and 1.\n"
        "Be objective and concise. Do NOT explain your reasoning."
        "Return ONLY a JSON object with keys: issue_type, confidence_score"
    )
)

issue_classifier_agent_result= issue_classifier_agent.run_sync(str(json_output))

# issue_classifier_agent_result_json_output = issue_classifier_agent_result.output.model_dump()
# print(json.dumps(issue_classifier_agent_result_json_output, indent=2))

parsed_output= IssueTypeOutput.model_validate_json(issue_classifier_agent_result.output)

# Now convert to dict and print
print(json.dumps(parsed_output.model_dump(), indent=2))

AttributeError: 'str' object has no attribute 'model_dump'

## Final test

In [ ]:
import nest_asyncio
nest_asyncio.apply()
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from pydantic_ai.models.groq import GroqModel
import json
from typing import Union
model = GroqModel('llama-3.1-8b-instant')

# ticket1 = {
#     "ticket_id": 12345,
#     "customer_tier": "premium",
#     "subject": "API returning 500 errors intermittently",
#     "message": "Hi, our production system has been failing",
#     "previous_tickets": 3,
#     "monthly_revenue": 5000,
#     "account_age_days": 450
# }

ticket1= {
"ticket_id": "SUP-005",
"customer_tier": "enterprise",
"subject": "Urgent: Security vulnerability?",
"message": "Our security team flagged that your API responses include internal server paths in danger",
"previous_tickets": 20,
"monthly_revenue": 50000,
"account_age_days": 900
}
class TicketAnalysis(BaseModel):
    summarized_issue: str= Field()
    sentiment: str = Field()
    urgency_level: str = Field()
    monthly_revenue: int
    account_age_days: int
    previous_tickets: int
    ticket_id: Union[str, int] 
    customer_tier: str

model= GroqModel("llama-3.1-8b-instant")

analyzer_agent= Agent(
    model=model,
    output_type=TicketAnalysis,
    system_prompt=(
        "You are a support ticket analyzer.\n"
        "Given a ticket with subject, message, revenue, and account age, perform the following:\n"
        "- Summarize the core issue in one short sentence explaining the issue to support.\n"
        "- Analyze the customer's tone to determine the sentiment (positive,or neutral, or negative).\n"
        "- Based on tone, content, and customer tier, assess urgency as Critical/P1, High/P2, Medium/P3, Low/P4. Determine urgency level. Even if sentiment is negative, consider customer tier and value. \n"
        "Return ONLY a JSON with: summarized_issue, sentiment, urgency_level, monthly_revenue, account_age_days, previous_tickets, ticket_id, customer_tier."
    )
)

analyzer_result= analyzer_agent.run_sync(str(ticket1))
intermediate_output= analyzer_result.output.model_dump()
print("\n[Agent 1 Output]")
print(json.dumps(intermediate_output, indent=2))

class IssueTypeOutput(BaseModel):
    issue_type: str= Field()
    confidence_score: float= Field()

issue_classifier_agent= Agent(
    model=model,
    output_type=IssueTypeOutput,
    system_prompt=(
        "You are an issue classification expert for customer support.\n"
        "Classify the issue based on the subject and message provided.\n"
        "Only use these categories: bug, feature_request, billing, security, login_issue, ui_feedback, documentation, general_query.\n"
        "Return a JSON with exactly two fields: issue_type (string), confidence_score (float between 0 and 1).\n"
        "Do not explain or add any other text. Output only a JSON object."
    )

)

classification_input = {
    "subject": ticket1["subject"],
    "message": ticket1["message"]
}
issue_classifier_result= issue_classifier_agent.run_sync(json.dumps(classification_input))

classified_issue= issue_classifier_result.output  
print("\n[Agent 2 Output]")
print(json.dumps(classified_issue.model_dump(), indent=2))
final_ticket_analysis = {
    **intermediate_output,
    **classified_issue.model_dump()
}

print("\n[Final Combined Output]")
print(json.dumps(final_ticket_analysis, indent=2))



[Agent 1 Output]
{
  "summarized_issue": "security vulnerability",
  "sentiment": "negative",
  "urgency_level": "Critical/P1",
  "monthly_revenue": 50000,
  "account_age_days": 900,
  "previous_tickets": 20,
  "ticket_id": "SUP-005",
  "customer_tier": "enterprise"
}

[Agent 2 Output]
{
  "issue_type": "security",
  "confidence_score": 0.8
}

[Final Combined Output]
{
  "summarized_issue": "security vulnerability",
  "sentiment": "negative",
  "urgency_level": "Critical/P1",
  "monthly_revenue": 50000,
  "account_age_days": 900,
  "previous_tickets": 20,
  "ticket_id": "SUP-005",
  "customer_tier": "enterprise",
  "issue_type": "security",
  "confidence_score": 0.8
}


In [59]:
intermediate_output

{'summarized_issue': 'security vulnerability',
 'sentiment': 'negative',
 'urgency_level': 'Critical/P1',
 'monthly_revenue': 50000,
 'account_age_days': 900,
 'previous_tickets': 20,
 'ticket_id': 'SUP-005',
 'customer_tier': 'enterprise'}

In [61]:
classified_issue.model_dump()

{'issue_type': 'security', 'confidence_score': 0.8}

In [62]:
ticket1

{'ticket_id': 'SUP-005',
 'customer_tier': 'enterprise',
 'subject': 'Urgent: Security vulnerability?',
 'message': 'Our security team flagged that your API responses include internal server paths in danger',
 'previous_tickets': 20,
 'monthly_revenue': 50000,
 'account_age_days': 900}